## Clustering Assignment
# Requirements
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

## Preprocessing

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import re

In [2]:
# getting data from internet
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print("Fetching Data From:\n",url,"\n========================")
source=requests.get(url).text
print("Completed")

Fetching Data From:
 https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 
Completed


## Processing-part-1: extracting raw table (from webpage)

In [3]:
soup=BeautifulSoup(source)
#print(soup.prettify())
table=soup.find("table",class_="wikitable sortable") 
#print(table.prettify())
rows=table.find_all("tr")
#print(rows)
l=[]
for row in rows:
    td=row.find_all("td")
    r=[d.text for d in td]
    l.append(r)
df=pd.DataFrame(l)

In [4]:
df.drop(0,inplace=True)  # as the first row contains the header information
df_col=["PostalCode","Borough","Neighborhood"]
df.columns=df_col
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


## Processing-part-2: integrating Postal codes with more than 1 neighbour

In [5]:
#In the Neighborhood column the information is ending with \n this needs to be removed. All the rows in the Borough column containing Not assigned must be deleted. If the Neighborhood is Not assigned then the Borough is assigned as the neighbourhood itself.
df["Neighborhood"]=df["Neighborhood"].str.replace("[\\\n]","")
df=df[df.Borough!="Not assigned"]
df.loc[df.Neighborhood=="Not assigned",["Neighborhood"]]=df.loc[df.Neighborhood=="Not assigned"].Borough
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


## Post-processing: creating an appropriate Pandas Dataframe

In [6]:
#Each Postal Code can contain multiple neighborhoods, so combining all the neighborhoods into single line seprated by ,. So there is no duplicity in the PostalCode i.e there is only one occurance of each postal code in the dataset
cln_df=df.groupby(["PostalCode","Borough"])['Neighborhood'].apply(lambda x:", ".join(x)).reset_index()
cln_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
cln_df.shape
print("There are {} rows in the Dataset.".format(cln_df.shape[0]))

There are 103 rows in the Dataset.
